In [1]:
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat

from Config_Data import load_config_data
from File_Operations import raw_text_from_file, write_point_to_docx, write_point_to_pdf
from Prompt import COMPLIANCE_CHECK_PROMPT, CORRECTION_PROMPT, WRITER_PROMPT

groq_api_key = load_config_data["GROQ_API_KEY"]
groq_model = load_config_data["GROQ_llama_3_3_70B_VERSATILE"]
groq_model_1 = load_config_data["GROQ_llama_3_3_70B_SPECTEC"]
groq_base_url = load_config_data["GROQ_BASE_URL"]
UPLOAD_DIR = load_config_data["FILE_PROCESSING_PATH"]

In [2]:
model_client = OpenAIChatCompletionClient(
        model = groq_model,
        api_key = groq_api_key,
        base_url = groq_base_url,
        temperature = 0.3,
        max_tokens = 512,
        model_info = {
            "vision": False,
            "function_calling": True,
            "json_output": False,
            "family": "unknown",
        },
    )

In [3]:
compliance_check_agent = AssistantAgent(
        name="Compliance_Check_Assistant",
        description="This agent is responsible for checking compliance in provided text.",
        system_message=COMPLIANCE_CHECK_PROMPT,
        model_client=model_client,
    )

In [4]:
compliance_correction_agent = AssistantAgent(
        name="Compliance_Correction_Assistant",
        description="Ensures text compliance with Federal Plain Language Guidelines by correcting grammar, sentence structure, and clarity.",
        system_message=CORRECTION_PROMPT,
        model_client=model_client,
    )

In [5]:
document_writer_agent = AssistantAgent(
        name = "Document_Writer_Agent",
        description = "Writes structured text to DOCX or PDF files based on function output without modifications.",
        system_message = WRITER_PROMPT,
        model_client = model_client,
        tools = [write_point_to_docx, write_point_to_pdf],
        reflect_on_tool_use = True,
    )

In [8]:
text = """ 
This rule proposes the Spring/Summer subsistence harvest regulations in Alaska for migratory birds that expire on August 31, 2003.

"""
stream = compliance_check_agent.run_stream(task=text)
await Console(stream)

---------- user ----------
 
This rule proposes the Spring/Summer subsistence harvest regulations in Alaska for migratory birds that expire on August 31, 2003.


---------- Compliance_Check_Assistant ----------
COMPLIANT


TaskResult(messages=[TextMessage(source='user', models_usage=None, content=' \nThis rule proposes the Spring/Summer subsistence harvest regulations in Alaska for migratory birds that expire on August 31, 2003.\n\n', type='TextMessage'), TextMessage(source='Compliance_Check_Assistant', models_usage=RequestUsage(prompt_tokens=397, completion_tokens=5), content='COMPLIANT', type='TextMessage')], stop_reason=None)

In [ ]:
agent_list = [compliance_correction_agent, document_writer_agent]
termination = TextMentionTermination("TERMINATE")
group_chat = RoundRobinGroupChat(agent_list, termination_condition=termination, max_turns=len(agent_list))

In [ ]:
input_text = f"""
text status: NON COMPLIANCE
file name: Test.pdf
text: {text}"""
stream = group_chat.run_stream(task=input_text)
await Console(stream)

In [ ]:
import requests, os

In [ ]:
def check_compliance(file_path:str):
    url = "http://127.0.0.1:8888/compliance_check"
    files = {"file": open(file_path, "rb")}
    response = requests.post(url, files=files)
    return response.json()

In [ ]:
file = r"C:\Users\laxmidhar.routa\OneDrive - Aspire Systems (India) Private Limited\Assessment\Document.docx"
print(check_compliance(file))

In [ ]:
def file_processing_api(file_path: str):
    url = "http://127.0.0.1:8888/updated_document_compliance"
    file_name, extension = os.path.splitext(os.path.basename(file_path))
    extension = extension.lower()
    if extension not in {".docx", ".pdf"}:
        print("Error: Unsupported file format. Only PDF and DOCX allowed.")
        return
    try:
        with open(file_path, "rb") as file:
            files = {"file": file}
            response = requests.post(url, files=files)
            response.raise_for_status()
            processed_file_path = f"processed_{file_name}{extension}"
            with open(processed_file_path, "wb") as f:
                f.write(response.content)
            print(f"Processed file downloaded successfully: {processed_file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error: Failed to process file. Details: {e}")
    except Exception as e:
        print(f"Unexpected Error: {e}")

In [ ]:
file_path = r"C:\Users\laxmidhar.routa\OneDrive - Aspire Systems (India) Private Limited\Assessment\Document.docx"
file_processing_api(file_path)